In [1]:
import json
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import ensemble, linear_model, metrics, model_selection

# fix numpy random seed
np.random.seed(666)


# First Exploration of Models


In [2]:
HYPER_max_pad = 10

os.makedirs("./results/stats", exist_ok=True)
categorical_vars = []
for i in range(HYPER_max_pad):
    categorical_vars.extend(
        [
            f"{x}_{i}"
            for x in (
                "Glasgow Coma Score - Verbal Response",
                "Glasgow Coma Score - Motor Response",
                "Glasgow Coma Score - Eye Opening",
                "Glasgow Coma Score - Total",
                "Circadian rhythm",
                "Richmond agitation-sedation scale",
                "Ventilator Airway Code",
            )
        ]
    )

models = (
    (
        "Logistic Regression",
        linear_model.LogisticRegression(random_state=666, n_jobs=-1),
    ),
    ("Random Forest", ensemble.RandomForestClassifier(random_state=666, n_jobs=-1)),
    ("Gradient Boosting", ensemble.GradientBoostingClassifier(random_state=666)),
)

# adapted from https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html
for model_name, model in models:
    for cl, cl_name in ((1, "Normal"), (2, "Mild"), (3, "Severe")):
        print(f"Model: {model_name} - Class: {cl_name}")
        X_train = pd.read_csv(
            f"./results/splits/X_train_{cl}_{cl_name}_padded_translated.csv",
            dtype={k: "category" for k in categorical_vars},
        )
        y_train = pd.read_csv(f"./results/splits/y_train_{cl}_{cl_name}.csv")
        X_test = pd.read_csv(
            f"./results/splits/X_test_{cl}_{cl_name}_padded_translated.csv",
            dtype={k: "category" for k in categorical_vars},
        )
        y_test = pd.read_csv(f"./results/splits/y_test_{cl}_{cl_name}.csv")
        X = X_train.append(other=X_test)
        y = y_train.append(other=y_test)

        # drop useless columns
        cols_to_drop = list()
        for col in X.columns:
            if (
                "Patient ID" in col  # remove all "Patient ID_N"
                or "Sequential ID" in col  # remove all "Sequential ID_N"
            ):
                cols_to_drop.append(col)
        X.drop(labels=cols_to_drop, axis=1, inplace=True)
        X = pd.get_dummies(data=X)

        cv = model_selection.StratifiedKFold(n_splits=5, random_state=666, shuffle=True)
        tprs = list()
        aucs = list()
        mean_fpr = np.linspace(0, 1, 100)

        fig, ax = plt.subplots(figsize=(20, 10))
        for i, (train, validation) in enumerate(cv.split(X, y)):
            model.fit(X=X.iloc[train], y=y.iloc[train].values.ravel())
            viz = metrics.RocCurveDisplay.from_estimator(
                estimator=model,
                X=X.iloc[validation],
                y=y.iloc[validation].values.ravel(),
                name=f"ROC fold {i}",
                pos_label=0,
                alpha=0.5,
                lw=2,
                ax=ax,
            )
            interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
            aucs.append(viz.roc_auc)

        ax.plot(
            [0, 1], [0, 1], linestyle="--", lw=4, color="r", label="Chance", alpha=0.8
        )

        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = metrics.auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
        ax.plot(
            mean_fpr,
            mean_tpr,
            color="b",
            label=f"Mean ROC (AUC = {mean_auc:0.2f} $\\pm$ {std_auc:0.2f})",
            lw=4,
            alpha=0.8,
        )

        std_tpr = np.std(tprs, axis=0)
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        ax.fill_between(
            mean_fpr,
            tprs_lower,
            tprs_upper,
            color="grey",
            alpha=0.2,
            label="$\\pm$ 1 std. dev.",
        )

        ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
        ax.set_title(f"Model: {model_name} - Class: {cl_name}", fontsize=35)
        ax.set_xlabel("1 - Specificity (FPR) (Positive label: 0)", fontsize=27)
        ax.set_ylabel("Sensitivity (TPR) (Positive label: 0)", fontsize=27)
        ax.set_xticklabels(
            labels=[-0.05] + [x / 10 for x in range(0, 12, 2)], fontsize=25
        )
        ax.set_yticklabels(
            labels=[-0.05] + [x / 10 for x in range(0, 12, 2)], fontsize=25
        )
        ax.legend(loc="lower right", fontsize=25)
        plt.tight_layout()
        plt.savefig(f"./results/stats/{model_name}_{cl}_{cl_name}.png")
        plt.close()


Model: Logistic Regression - Class: Normal


D:\Windows\TEMP/ipykernel_19252/1367060865.py:114: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(
D:\Windows\TEMP/ipykernel_19252/1367060865.py:117: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(


Model: Logistic Regression - Class: Mild


D:\Windows\TEMP/ipykernel_19252/1367060865.py:114: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(
D:\Windows\TEMP/ipykernel_19252/1367060865.py:117: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(


Model: Logistic Regression - Class: Severe


...

D:\Windows\TEMP/ipykernel_19252/1367060865.py:114: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(
D:\Windows\TEMP/ipykernel_19252/1367060865.py:117: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(


Model: Gradient Boosting - Class: Mild


D:\Windows\TEMP/ipykernel_19252/1367060865.py:114: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(
D:\Windows\TEMP/ipykernel_19252/1367060865.py:117: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(


Model: Gradient Boosting - Class: Severe


D:\Windows\TEMP/ipykernel_19252/1367060865.py:114: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(
D:\Windows\TEMP/ipykernel_19252/1367060865.py:117: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(


# Tuning HyperParameters


In [3]:
HYPER_max_pad = 10

categorical_vars = []
for i in range(HYPER_max_pad):
    categorical_vars.extend(
        [
            f"{x}_{i}"
            for x in (
                "Glasgow Coma Score - Verbal Response",
                "Glasgow Coma Score - Motor Response",
                "Glasgow Coma Score - Eye Opening",
                "Glasgow Coma Score - Total",
                "Circadian rhythm",
                "Richmond agitation-sedation scale",
                "Ventilator Airway Code",
            )
        ]
    )

models = (
    (
        "Logistic Regression",
        linear_model.LogisticRegression(class_weight="balanced", random_state=666),
        dict(penalty=["l2", "none"], max_iter=[100, 3000, 10000]),
    ),
    (
        "Random Forest",
        ensemble.RandomForestClassifier(criterion="gini", random_state=666),
        dict(
            n_estimators=[100, 500, 2500],
            class_weight=["balanced", "balanced_subsample"],
        ),
    ),
    (
        "Gradient Boosting",
        ensemble.GradientBoostingClassifier(random_state=666),
        dict(
            learning_rate=[0.1, 0.05], n_estimators=[100, 500, 2500], max_depth=[3, 7]
        ),
    ),
)

results = dict()
for model_name, model, hyperparameters in models:
    for cl, cl_name in ((1, "Normal"), (2, "Mild"), (3, "Severe")):
        print(f"Model: {model_name} - Class: {cl_name}")
        X_train = pd.read_csv(
            f"./results/splits/X_train_{cl}_{cl_name}_padded_translated.csv",
            dtype={k: "category" for k in categorical_vars},
        )
        y_train = pd.read_csv(f"./results/splits/y_train_{cl}_{cl_name}.csv")

        # drop useless columns
        cols_to_drop = list()
        for col in X_train.columns:
            if (
                "Patient ID" in col  # remove all "Patient ID_N"
                or "Sequential ID" in col  # remove all "Sequential ID_N"
            ):
                cols_to_drop.append(col)
        X_train.drop(labels=cols_to_drop, axis=1, inplace=True)
        X_train = pd.get_dummies(X_train)

        cv = model_selection.StratifiedKFold(n_splits=5, random_state=666, shuffle=True)
        clf = model_selection.GridSearchCV(
            estimator=model,
            param_grid=hyperparameters,
            scoring=metrics.make_scorer(metrics.matthews_corrcoef),
            n_jobs=-1,
            cv=cv,
        )
        results[f"{model_name}_{cl}_{cl_name}"] = clf.fit(
            X=X_train, y=y_train.values.ravel()
        )

for k, v in results.items():
    print(f"{k}:\n\tBest score: {v.best_score_}\n\tBest parameters: {v.best_params_}")

# save some data of the hyperparameters' search
with open("./results/best_models.json", "w") as f:
    json.dump(
        {
            k: dict(
                param_grid=v.param_grid,
                best_index=int(v.best_index_),
                best_score_=float(v.best_score_),
                best_params_=v.best_params_,
                mean_fit_time=[float(x) for x in v.cv_results_["mean_fit_time"]],
                std_fit_time=[float(x) for x in v.cv_results_["std_fit_time"]],
                std_score_time=[float(x) for x in v.cv_results_["std_score_time"]],
                params=v.cv_results_["params"],
                split0_test_score=[
                    float(x) for x in v.cv_results_["split0_test_score"]
                ],
                split1_test_score=[
                    float(x) for x in v.cv_results_["split1_test_score"]
                ],
                split2_test_score=[
                    float(x) for x in v.cv_results_["split2_test_score"]
                ],
                split3_test_score=[
                    float(x) for x in v.cv_results_["split3_test_score"]
                ],
                split4_test_score=[
                    float(x) for x in v.cv_results_["split4_test_score"]
                ],
                mean_test_score=[float(x) for x in v.cv_results_["mean_test_score"]],
                std_test_score=[float(x) for x in v.cv_results_["std_test_score"]],
                rank_test_score=[float(x) for x in v.cv_results_["rank_test_score"]],
            )
            for k, v in results.items()
        },
        f,
        indent=4,
    )


Model: Logistic Regression - Class: Normal


d:\Nextcloud\Education\Data Science\Thesis\Code\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: Logistic Regression - Class: Mild


d:\Nextcloud\Education\Data Science\Thesis\Code\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: Logistic Regression - Class: Severe


d:\Nextcloud\Education\Data Science\Thesis\Code\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: Random Forest - Class: Normal
Model: Random Forest - Class: Mild
Model: Random Forest - Class: Severe
Model: Gradient Boosting - Class: Normal
Model: Gradient Boosting - Class: Mild
Model: Gradient Boosting - Class: Severe
Logistic Regression_1_Normal:
	Best score: 0.2228493386094314
	Best parameters: {'max_iter': 10000, 'penalty': 'l2'}
Logistic Regression_2_Mild:
	Best score: 0.3749702151157529
	Best parameters: {'max_iter': 10000, 'penalty': 'l2'}
Logistic Regression_3_Severe:
	Best score: 0.49927352749802634
	Best parameters: {'max_iter': 10000, 'penalty': 'none'}
Random Forest_1_Normal:
	Best score: 0.028239483710008107
	Best parameters: {'class_weight': 'balanced_subsample', 'n_estimators': 100}
Random Forest_2_Mild:
	Best score: 0.3178088054657583
	Best parameters: {'class_weight': 'balanced_subsample', 'n_estimators': 100}
Random Forest_3_Severe:
	Best score: 0.4074559118436597
	Best parameters: {'class_weight': 'balanced', 'n_estimators': 100}
Gradient Boosting_1_Normal

# Select Best Model per Class


In [4]:
best_models = None
with open("./results/best_models.json", "r") as f:
    best_models = json.load(f)

models = dict()
for cl in ("Normal", "Mild", "Severe"):
    print(cl)
    for k, v in filter(lambda k: cl in k[0], best_models.items()):
        print(f"\t{k}: {v['best_score_']}\n\tParameters: {v['best_params_']}")


Normal
	Logistic Regression_1_Normal: 0.2228493386094314
	Parameters: {'max_iter': 10000, 'penalty': 'l2'}
	Random Forest_1_Normal: 0.028239483710008107
	Parameters: {'class_weight': 'balanced_subsample', 'n_estimators': 100}
	Gradient Boosting_1_Normal: 0.12997772129113114
	Parameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 2500}
Mild
	Logistic Regression_2_Mild: 0.3749702151157529
	Parameters: {'max_iter': 10000, 'penalty': 'l2'}
	Random Forest_2_Mild: 0.3178088054657583
	Parameters: {'class_weight': 'balanced_subsample', 'n_estimators': 100}
	Gradient Boosting_2_Mild: 0.39283898313471344
	Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
Severe
	Logistic Regression_3_Severe: 0.49927352749802634
	Parameters: {'max_iter': 10000, 'penalty': 'none'}
	Random Forest_3_Severe: 0.4074559118436597
	Parameters: {'class_weight': 'balanced', 'n_estimators': 100}
	Gradient Boosting_3_Severe: 0.511573650582509
	Parameters: {'learning_rate': 0.05, 'max_depth':

# Evaluation of the Models


In [5]:
HYPER_max_pad = 10

os.makedirs("./results/stats", exist_ok=True)
categorical_vars = []
for i in range(HYPER_max_pad):
    categorical_vars.extend(
        [
            f"{x}_{i}"
            for x in (
                "Glasgow Coma Score - Verbal Response",
                "Glasgow Coma Score - Motor Response",
                "Glasgow Coma Score - Eye Opening",
                "Glasgow Coma Score - Total",
                "Circadian rhythm",
                "Richmond agitation-sedation scale",
                "Ventilator Airway Code",
            )
        ]
    )

best_models = None
with open("./results/best_models.json", "r") as f:
    best_models = json.load(f)

models = (
    (
        "Logistic Regression_1_Normal",
        linear_model.LogisticRegression(
            **best_models["Logistic Regression_1_Normal"]["best_params_"],
            random_state=666,
            n_jobs=-1,
        ),
    ),
    (
        "Gradient Boosting_2_Mild",
        ensemble.GradientBoostingClassifier(
            **best_models["Gradient Boosting_2_Mild"]["best_params_"], random_state=666
        ),
    ),
    (
        "Gradient Boosting_3_Severe",
        ensemble.GradientBoostingClassifier(
            **best_models["Gradient Boosting_3_Severe"]["best_params_"],
            random_state=666,
        ),
    ),
)

stats_columns = [
    "Model_Class",
    "Iteration",
    "#True Positives",
    "#False Negatives",
    "#False Positives",
    "#True Negatives",
    "False Positive Rates",
    "True Positive Rates",
    "Interpolation FPR-TPR",
    "ROC-AUC",
    "No Skill",
    "Precisions",
    "Recalls",
    "Interpolation P-R",
    "PR-AUC",
]
df_metrics = pd.DataFrame(columns=stats_columns)
# adapted from https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html
for model_name, model in models:
    model_name, cl, cl_name = model_name.split("_")
    print(f"Model: {model_name} - Class: {cl_name}")
    X_train = pd.read_csv(
        f"./results/splits/X_train_{cl}_{cl_name}_padded_translated.csv",
        dtype={k: "category" for k in categorical_vars},
    )
    y_train = pd.read_csv(f"./results/splits/y_train_{cl}_{cl_name}.csv")

    # drop useless columns
    cols_to_drop = list()
    for col in X_train.columns:
        if (
            "Patient ID" in col  # remove all "Patient ID_N"
            or "Sequential ID" in col  # remove all "Sequential ID_N"
        ):
            cols_to_drop.append(col)
    X_train.drop(labels=cols_to_drop, axis=1, inplace=True)
    X_train = pd.get_dummies(data=X_train)

    cv = model_selection.StratifiedKFold(n_splits=5, random_state=666, shuffle=True)
    fig1, ax1 = plt.subplots(figsize=(20, 10))
    fig2, ax2 = plt.subplots(figsize=(20, 10))
    fig3, ax3 = plt.subplots(figsize=(20, 10))
    for i, (train, validation) in enumerate(cv.split(X_train, y_train)):
        model.fit(X=X_train.iloc[train], y=y_train.iloc[train].values.ravel())
        cm_display = metrics.ConfusionMatrixDisplay.from_estimator(
            estimator=model,
            X=X_train.iloc[validation],
            y=y_train.iloc[validation].values.ravel(),
            ax=ax1,
        )
        pr_display = metrics.PrecisionRecallDisplay.from_estimator(
            estimator=model,
            X=X_train.iloc[validation],
            y=y_train.iloc[validation].values.ravel(),
            name=f"PR fold {i}",
            pos_label=0,
            alpha=0.5,
            lw=2,
            ax=ax2,
        )
        roc_display = metrics.RocCurveDisplay.from_estimator(
            estimator=model,
            X=X_train.iloc[validation],
            y=y_train.iloc[validation].values.ravel(),
            name=f"ROC fold {i}",
            pos_label=0,
            alpha=0.5,
            lw=2,
            ax=ax3,
        )
        interpolated_fpr_tpr = np.interp(
            np.linspace(0, 1, 100),  # interpolation range
            roc_display.fpr,  # x
            roc_display.tpr,  # y
        )
        interpolated_fpr_tpr[0] = 0.0
        interpolated_pr_rec = np.interp(
            np.linspace(0, 1, 100),  # interpolation range
            pr_display.precision,  # x
            pr_display.recall,  # y
        )
        interpolated_pr_rec[0] = 0.0
        df_metrics = df_metrics.append(
            pd.DataFrame(
                [
                    [
                        f"{model_name}_{cl}_{cl_name}",  # model_class_class_name
                        i,  # iteration
                        cm_display.confusion_matrix[0][0],  # tp
                        cm_display.confusion_matrix[0][1],  # fn
                        cm_display.confusion_matrix[1][0],  # fp
                        cm_display.confusion_matrix[1][1],  # tn
                        ",".join(
                            f"{float(x)}" for x in roc_display.fpr
                        ),  # comma-separated values
                        ",".join(
                            f"{float(x)}" for x in roc_display.tpr
                        ),  # comma-separated values
                        ",".join(
                            f"{float(x)}" for x in interpolated_fpr_tpr
                        ),  # comma-separated values
                        roc_display.roc_auc,  # roc auc
                        len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
                        / len(y_train.iloc[validation]),  # no skill
                        ",".join(
                            f"{float(x)}" for x in pr_display.precision
                        ),  # comma-separated values
                        ",".join(
                            f"{float(x)}" for x in pr_display.recall
                        ),  # comma-separated values
                        ",".join(
                            f"{float(x)}" for x in interpolated_pr_rec
                        ),  # comma-separated values
                        pr_display.average_precision,
                    ]
                ],
                columns=stats_columns,
            )
        )

    tprs = [
        [float(tpr) for tpr in tprs_i.split(",")]
        for tprs_i in df_metrics[
            df_metrics["Model_Class"] == f"{model_name}_{cl}_{cl_name}"
        ]["Interpolation FPR-TPR"]
    ]
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = metrics.auc(np.linspace(0, 1, 100), mean_tpr)
    rocs = [
        roc_auc
        for roc_auc in df_metrics[
            df_metrics["Model_Class"] == f"{model_name}_{cl}_{cl_name}"
        ]["ROC-AUC"]
    ]
    std_auc = np.std(rocs)
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)

    ax3.plot([0, 1], [0, 1], linestyle="--", lw=4, color="r", label="Chance", alpha=0.8)
    ax3.plot(
        np.linspace(0, 1, 100),
        mean_tpr,
        color="b",
        label=f"Mean ROC (AUC = {mean_auc:0.2f} $\\pm$ {std_auc:0.2f})",
        lw=4,
        alpha=0.8,
    )
    ax3.fill_between(
        np.linspace(0, 1, 100),
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label="$\\pm$ 1 std. dev.",
    )

    ax3.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
    ax3.set_title(f"Model: {model_name} - Class: {cl_name}", fontsize=35)
    ax3.set_xlabel("1 - Specificity (FPR) (Positive label: 0)", fontsize=27)
    ax3.set_ylabel("Sensitivity (TPR) (Positive label: 0)", fontsize=27)
    ax3.set_xticklabels(labels=[-0.05] + [x / 10 for x in range(0, 12, 2)], fontsize=25)
    ax3.set_yticklabels(labels=[-0.05] + [x / 10 for x in range(0, 12, 2)], fontsize=25)
    ax3.legend(loc="lower right", fontsize=25)
    plt.tight_layout()
    fig2.savefig(f"./results/stats/tuned_{model_name}_{cl}_{cl_name}_prauc.png")
    fig3.savefig(f"./results/stats/tuned_{model_name}_{cl}_{cl_name}_rocauc.png")
    plt.close("all")

df_metrics["Accuracy"] = (
    df_metrics["#True Positives"] + df_metrics["#True Negatives"]
) / (
    df_metrics["#True Positives"]
    + df_metrics["#True Negatives"]
    + df_metrics["#False Positives"]
    + df_metrics["#False Negatives"]
).replace(
    {0: np.nan}
)
# positive predictive value = precision
df_metrics["Positive Predictive Value"] = df_metrics["#True Positives"] / (
    df_metrics["#True Positives"] + df_metrics["#False Positives"]
).replace({0: np.nan})
df_metrics["Negative Predictive Value"] = df_metrics["#True Negatives"] / (
    df_metrics["#True Negatives"] + df_metrics["#False Negatives"]
).replace({0: np.nan})
# sensitivity = recall
df_metrics["Sensitivity"] = df_metrics["#True Positives"] / (
    df_metrics["#True Positives"] + df_metrics["#False Negatives"]
).replace({0: np.nan})
df_metrics["Specificity"] = df_metrics["#True Negatives"] / (
    df_metrics["#True Negatives"] + df_metrics["#False Positives"]
).replace({0: np.nan})
df_metrics["F1"] = (
    2
    * (df_metrics["Positive Predictive Value"] * df_metrics["Sensitivity"])
    / (df_metrics["Positive Predictive Value"] + df_metrics["Sensitivity"]).replace(
        {0: np.nan}
    )
)
df_metrics["Matthews Correlation Coefficient"] = (
    (df_metrics["#True Positives"] * df_metrics["#True Negatives"])
    - (df_metrics["#False Positives"] * df_metrics["#False Negatives"])
) / (
    (
        (df_metrics["#True Positives"] + df_metrics["#False Positives"])
        * (df_metrics["#True Positives"] + df_metrics["#False Negatives"])
        * (df_metrics["#True Negatives"] + df_metrics["#False Positives"])
        * (df_metrics["#True Negatives"] + df_metrics["#False Negatives"])
    ).replace({0: np.nan})
    ** (1 / 2)  # sqrt
)
df_metrics.to_csv("./results/stats/cv_metrics.csv", index=False)


Model: Logistic Regression - Class: Normal


D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:

Model: Gradient Boosting - Class: Mild


D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:

Model: Gradient Boosting - Class: Severe


D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:152: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(y_train.iloc[validation][y_train["Lactate_Outcome"] == 1])
D:\Windows\TEMP/ipykernel_19252/601780011.py:

In [6]:
df_metrics = pd.read_csv("./results/stats/cv_metrics.csv")
for cl_name in ("Normal", "Mild", "Severe"):
    print(cl_name)
    df_tmp = df_metrics[df_metrics["Model_Class"].str.contains(cl_name)]
    for col in df_tmp.columns[2:]:
        try:
            print(
                f"{col}: Mean: {np.nanmean(df_tmp[col]):0.2f}, STD: {np.nanstd(df_tmp[col]):0.2f}"
            )
        except Exception as ex:
            print(f"Error while computing mean and stdev for {col}")


Normal
#True Positives: Mean: 7840.00, STD: 4.86
#False Negatives: Mean: 19.00, STD: 4.86
#False Positives: Mean: 695.20, STD: 1.72
#True Negatives: Mean: 9.00, STD: 1.90
Error while computing mean and stdev for False Positive Rates
Error while computing mean and stdev for True Positive Rates
Error while computing mean and stdev for Interpolation FPR-TPR
ROC-AUC: Mean: 0.75, STD: 0.01
No Skill: Mean: 0.08, STD: 0.00
Error while computing mean and stdev for Precisions
Error while computing mean and stdev for Recalls
Error while computing mean and stdev for Interpolation P-R
PR-AUC: Mean: 0.97, STD: 0.00
Accuracy: Mean: 0.92, STD: 0.00
Positive Predictive Value: Mean: 0.92, STD: 0.00
Negative Predictive Value: Mean: 0.33, STD: 0.06
Sensitivity: Mean: 1.00, STD: 0.00
Specificity: Mean: 0.01, STD: 0.00
F1: Mean: 0.96, STD: 0.00
Matthews Correlation Coefficient: Mean: 0.05, STD: 0.01
Mild
#True Positives: Mean: 681.60, STD: 12.47
#False Negatives: Mean: 544.80, STD: 12.27
#False Positives: 

# Final Models


In [7]:
HYPER_max_pad = 10

os.makedirs("./results/stats", exist_ok=True)
categorical_vars = []
for i in range(HYPER_max_pad):
    categorical_vars.extend(
        [
            f"{x}_{i}"
            for x in (
                "Glasgow Coma Score - Verbal Response",
                "Glasgow Coma Score - Motor Response",
                "Glasgow Coma Score - Eye Opening",
                "Glasgow Coma Score - Total",
                "Circadian rhythm",
                "Richmond agitation-sedation scale",
                "Ventilator Airway Code",
            )
        ]
    )

best_models = None
with open("./results/best_models.json", "r") as f:
    best_models = json.load(f)

models = (
    (
        "Logistic Regression_1_Normal",
        linear_model.LogisticRegression(
            **best_models["Logistic Regression_1_Normal"]["best_params_"],
            random_state=666,
            n_jobs=-1,
        ),
    ),
    (
        "Gradient Boosting_2_Mild",
        ensemble.GradientBoostingClassifier(
            **best_models["Gradient Boosting_2_Mild"]["best_params_"], random_state=666
        ),
    ),
    (
        "Gradient Boosting_3_Severe",
        ensemble.GradientBoostingClassifier(
            **best_models["Gradient Boosting_3_Severe"]["best_params_"],
            random_state=666,
        ),
    ),
)

stats_columns = [
    "Model_Class",
    "#True Positives",
    "#False Negatives",
    "#False Positives",
    "#True Negatives",
    "False Positive Rates",
    "True Positive Rates",
    "Interpolation FPR-TPR",
    "ROC-AUC",
    "No Skill",
    "Precisions",
    "Recalls",
    "Interpolation P-R",
    "PR-AUC",
]
df_metrics = pd.DataFrame(columns=stats_columns)
# adapted from https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html
for model_name, model in models:
    model_name, cl, cl_name = model_name.split("_")
    print(f"Model: {model_name} - Class: {cl_name}")
    X_train = pd.read_csv(
        f"./results/splits/X_train_{cl}_{cl_name}_padded_translated.csv",
        dtype={k: "category" for k in categorical_vars},
    )
    y_train = pd.read_csv(f"./results/splits/y_train_{cl}_{cl_name}.csv")
    X_test = pd.read_csv(
        f"./results/splits/X_test_{cl}_{cl_name}_padded_translated.csv",
        dtype={k: "category" for k in categorical_vars},
    )
    y_test = pd.read_csv(f"./results/splits/y_test_{cl}_{cl_name}.csv")

    # drop useless columns
    cols_to_drop = list()
    for col in X_train.columns:
        if (
            "Patient ID" in col  # remove all "Patient ID_N"
            or "Sequential ID" in col  # remove all "Sequential ID_N"
        ):
            cols_to_drop.append(col)
    X_train.drop(labels=cols_to_drop, axis=1, inplace=True)
    X_test.drop(labels=cols_to_drop, axis=1, inplace=True)
    X = X_train.append(other=X_test)
    X = pd.get_dummies(X)
    X_train = X[: len(X_train)]
    X_test = X[len(X_train) :]

    fig1, ax1 = plt.subplots(figsize=(20, 10))
    fig2, ax2 = plt.subplots(figsize=(20, 10))
    fig3, ax3 = plt.subplots(figsize=(20, 10))
    model.fit(X=X_train, y=y_train.values.ravel())
    cm_display = metrics.ConfusionMatrixDisplay.from_estimator(
        estimator=model, X=X_test, y=y_test, ax=ax1
    )
    pr_display = metrics.PrecisionRecallDisplay.from_estimator(
        estimator=model, X=X_test, y=y_test, name=f"PR", pos_label=0, ax=ax2
    )
    roc_display = metrics.RocCurveDisplay.from_estimator(
        estimator=model, X=X_test, y=y_test, name=f"ROC", pos_label=0, ax=ax3
    )
    interpolated_fpr_tpr = np.interp(
        np.linspace(0, 1, 100),  # interpolation range
        roc_display.fpr,  # x
        roc_display.tpr,  # y
    )
    interpolated_fpr_tpr[0] = 0.0
    interpolated_pr_rec = np.interp(
        np.linspace(0, 1, 100),  # interpolation range
        pr_display.precision,  # x
        pr_display.recall,  # y
    )
    interpolated_pr_rec[0] = 0.0
    df_metrics = df_metrics.append(
        pd.DataFrame(
            [
                [
                    f"{model_name}_{cl}_{cl_name}",  # model_class_class_name
                    cm_display.confusion_matrix[0][0],  # tp
                    cm_display.confusion_matrix[0][1],  # fn
                    cm_display.confusion_matrix[1][0],  # fp
                    cm_display.confusion_matrix[1][1],  # tn
                    ",".join(
                        f"{float(x)}" for x in roc_display.fpr
                    ),  # comma-separated values
                    ",".join(
                        f"{float(x)}" for x in roc_display.tpr
                    ),  # comma-separated values
                    ",".join(
                        f"{float(x)}" for x in interpolated_fpr_tpr
                    ),  # comma-separated values
                    roc_display.roc_auc,  # roc auc
                    len(y_test[y_test["Lactate_Outcome"] == 1])
                    / len(y_test),  # no skill
                    ",".join(
                        f"{float(x)}" for x in pr_display.precision
                    ),  # comma-separated values
                    ",".join(
                        f"{float(x)}" for x in pr_display.recall
                    ),  # comma-separated values
                    ",".join(
                        f"{float(x)}" for x in interpolated_pr_rec
                    ),  # comma-separated values
                    pr_display.average_precision,
                ]
            ],
            columns=stats_columns,
        )
    )
    ax3.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

    ax3.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
    ax3.set_title(f"Model: {model_name} - Class: {cl_name}", fontsize=35)
    ax3.set_xlabel("1 - Specificity (FPR) (Positive label: 0)", fontsize=27)
    ax3.set_ylabel("Sensitivity (TPR) (Positive label: 0)", fontsize=27)
    ax3.set_xticklabels(labels=[-0.05] + [x / 10 for x in range(0, 12, 2)], fontsize=25)
    ax3.set_yticklabels(labels=[-0.05] + [x / 10 for x in range(0, 12, 2)], fontsize=25)
    ax3.legend(loc="lower right", fontsize=25)
    plt.tight_layout()
    fig1.savefig(f"./results/stats/final_{model_name}_{cl}_{cl_name}_confmat.png")
    fig2.savefig(f"./results/stats/final_{model_name}_{cl}_{cl_name}_prauc.png")
    fig3.savefig(f"./results/stats/final_{model_name}_{cl}_{cl_name}_rocauc.png")
    plt.close("all")


df_metrics["Accuracy"] = (
    df_metrics["#True Positives"] + df_metrics["#True Negatives"]
) / (
    df_metrics["#True Positives"]
    + df_metrics["#True Negatives"]
    + df_metrics["#False Positives"]
    + df_metrics["#False Negatives"]
)
# positive predictive value = precision
df_metrics["Positive Predictive Value"] = df_metrics["#True Positives"] / (
    df_metrics["#True Positives"] + df_metrics["#False Positives"]
)
df_metrics["Negative Predictive Value"] = df_metrics["#True Negatives"] / (
    df_metrics["#True Negatives"] + df_metrics["#False Negatives"]
)
# sensitivity = recall
df_metrics["Sensitivity"] = df_metrics["#True Positives"] / (
    df_metrics["#True Positives"] + df_metrics["#False Negatives"]
)
df_metrics["Specificity"] = df_metrics["#True Negatives"] / (
    df_metrics["#True Negatives"] + df_metrics["#False Positives"]
)
df_metrics["F1"] = (
    2
    * (df_metrics["Positive Predictive Value"] * df_metrics["Sensitivity"])
    / (df_metrics["Positive Predictive Value"] + df_metrics["Sensitivity"])
)
df_metrics["Matthews Correlation Coefficient"] = (
    (df_metrics["#True Positives"] * df_metrics["#True Negatives"])
    - (df_metrics["#False Positives"] * df_metrics["#False Negatives"])
) / (
    (
        (df_metrics["#True Positives"] + df_metrics["#False Positives"])
        * (df_metrics["#True Positives"] + df_metrics["#False Negatives"])
        * (df_metrics["#True Negatives"] + df_metrics["#False Positives"])
        * (df_metrics["#True Negatives"] + df_metrics["#False Negatives"])
    )
    ** (1 / 2)  # sqrt
)
df_metrics.to_csv("./results/stats/final_metrics.csv", index=False)


Model: Logistic Regression - Class: Normal


D:\Windows\TEMP/ipykernel_19252/1341005931.py:163: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax3.set_xticklabels(labels=[-0.05] + [x / 10 for x in range(0, 12, 2)], fontsize=25)
D:\Windows\TEMP/ipykernel_19252/1341005931.py:164: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax3.set_yticklabels(labels=[-0.05] + [x / 10 for x in range(0, 12, 2)], fontsize=25)


Model: Gradient Boosting - Class: Mild


D:\Windows\TEMP/ipykernel_19252/1341005931.py:163: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax3.set_xticklabels(labels=[-0.05] + [x / 10 for x in range(0, 12, 2)], fontsize=25)
D:\Windows\TEMP/ipykernel_19252/1341005931.py:164: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax3.set_yticklabels(labels=[-0.05] + [x / 10 for x in range(0, 12, 2)], fontsize=25)


Model: Gradient Boosting - Class: Severe


D:\Windows\TEMP/ipykernel_19252/1341005931.py:163: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax3.set_xticklabels(labels=[-0.05] + [x / 10 for x in range(0, 12, 2)], fontsize=25)
D:\Windows\TEMP/ipykernel_19252/1341005931.py:164: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax3.set_yticklabels(labels=[-0.05] + [x / 10 for x in range(0, 12, 2)], fontsize=25)


In [8]:
df_metrics = pd.read_csv("./results/stats/final_metrics.csv")
for col in df_metrics.columns[1:]:
    try:
        print(
            f"{col}: Mean: {np.nanmean(df_metrics[col]):0.2f}, STD: {np.nanstd(df_metrics[col]):0.2f}"
        )
    except Exception as ex:
        print(f"Error while computing mean and stdev for {col}")


#True Positives: Mean: 4863.33, STD: 5770.77
#False Negatives: Mean: 405.33, STD: 365.79
#False Positives: Mean: 698.00, STD: 405.07
#True Negatives: Mean: 1576.33, STD: 1196.15
Error while computing mean and stdev for False Positive Rates
Error while computing mean and stdev for True Positive Rates
Error while computing mean and stdev for Interpolation FPR-TPR
ROC-AUC: Mean: 0.80, STD: 0.05
No Skill: Mean: 0.48, STD: 0.28
Error while computing mean and stdev for Precisions
Error while computing mean and stdev for Recalls
Error while computing mean and stdev for Interpolation P-R
PR-AUC: Mean: 0.76, STD: 0.14
Accuracy: Mean: 0.81, STD: 0.08
Positive Predictive Value: Mean: 0.74, STD: 0.12
Negative Predictive Value: Mean: 0.69, STD: 0.17
Sensitivity: Mean: 0.72, STD: 0.20
Specificity: Mean: 0.57, STD: 0.40
F1: Mean: 0.73, STD: 0.16
Matthews Correlation Coefficient: Mean: 0.32, STD: 0.19
